In [1]:
import cudf as cd
import cupy as cp
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
dftrain1=cd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
dftest1=cd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [4]:
dftrain1.head()

In [5]:
#Preprocessing Training data
dftrain1.dtypes

In [6]:
dftrain1.info()

In [7]:
toprocess=[]
for i in dftrain1:
    if dftrain1[i].isnull().sum() > 0:
        toprocess.append(i)
toprocess

In [8]:
inttoprocess=dftrain1[toprocess].select_dtypes(include=['float64']).columns
cattoprocess=dftrain1[toprocess].select_dtypes(include=['object']).columns

In [9]:
for i in inttoprocess:
    dftrain1[i]=dftrain1[i].fillna(dftrain1[i].mean())

In [10]:
for i in cattoprocess:
    dftrain1[i]=dftrain1[i].fillna(method='ffill')

In [11]:
#No Null values in Training Data
toprocess1=[]
for i in dftrain1:
    if dftrain1[i].isnull().sum() > 0:
        toprocess1.append(i)
toprocess1

In [12]:
toprocesstest=[]
for i in dftest1:
    if dftest1[i].isnull().sum() > 0:
        toprocesstest.append(i)
toprocesstest

In [13]:
inttoprocesstest=dftest1[toprocesstest].select_dtypes(include=['float64']).columns
cattoprocesstest=dftest1[toprocesstest].select_dtypes(include=['object']).columns

In [14]:
for i in inttoprocesstest:
    dftest1[i]=dftest1[i].fillna(dftest1[i].mean())

In [17]:
for i in cattoprocesstest:
    dftest1[i]=dftest1[i].fillna(method='bfill')

In [18]:
#No Null Values now in Test Data
toprocess1test=[]
for i in dftest1:
    if dftest1[i].isnull().sum() > 0:
        toprocess1test.append(i)
toprocess1test

In [19]:
catdata=dftrain1.select_dtypes(include=['object']).columns
catdata

In [20]:
#Encoding Categorical Data in Training Data
dftrain1=cd.get_dummies(dftrain1,columns=catdata)
dftrain1

In [21]:
dftrain1.info()

In [22]:
#All Categorical Encoded Values in Training Data
catdat1a=dftrain1.select_dtypes(include=['object']).columns
catdat1a

In [23]:
#Encoding Categorical Values in Test Data
cattestdata=dftest1.select_dtypes(include=['object']).columns
dftest1=cd.get_dummies(dftest1,columns=cattestdata)
dftest1

In [24]:
#All Categorical Encoded Values in Test Data
catdata1test=dftest1.select_dtypes(include=['object']).columns
catdata1test

In [25]:
a=dftrain1.columns.intersection(dftest1.columns)
b=dftest1.columns.intersection(dftrain1.columns)
dftrain=dftrain1[a]
dftest=dftest1[b]
dftest=dftest.drop(['Id'],axis=1)
dftrain=dftrain.drop(['Id'],axis=1)
dftrain['SalePrice']=dftrain1['SalePrice']

In [26]:
dftest

In [37]:
#Scaling Features
from cuml.preprocessing import StandardScaler
ss=StandardScaler()
totrain=dftrain.drop('SalePrice',axis=1)
sdftrain=ss.fit_transform(totrain)
sdftest=ss.fit_transform(dftest)

In [28]:
dftest.select_dtypes(include=['object']).columns

In [29]:
dftrain.columns

In [30]:
#Model Training on df_train
x=sdftrain
y=dftrain['SalePrice']

In [51]:
from cuml import LinearRegression
lr=LinearRegression(fit_intercept = True, normalize = False,algorithm = 'eig')
reg = lr.fit(x,y)
print("Coefficients:")
print(reg.coef_)
print("Intercept:")
print(reg.intercept_)

In [52]:
ypred=lr.predict(sdftest)
ypred

In [53]:
idd=list(range(1461,2920))
ypredlist=ypred.to_arrow().to_pylist()
data={'Id':idd,'SalePrice':ypredlist}
resdf=cd.DataFrame(data)

In [54]:
resdf

In [ ]:
resdf.to_csv('Submission.csv',columns=['Id','SalePrice'],index=False)